# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [14]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import geoviews as gv
import requests
!pip install citipy
from citipy import citipy

# Import API key
from api_keys import geoapify_key

In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,colchani,-20.3000,-66.9333,50.0,33,96,4.92,BO,2024-06-08
1,1,qaqortoq,60.7167,-46.0333,39.2,98,100,1.09,GL,2024-06-08
2,2,adamstown,-25.0660,-130.1015,73.4,82,11,4.97,PN,2024-06-08
3,3,happy valley-goose bay,53.3168,-60.3315,69.8,46,6,2.86,CA,2024-06-08
4,4,mogocha,53.7333,119.7667,42.8,60,6,1.31,RU,2024-06-08


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

map_plot = city_data_df.hvplot.points(
    'Lng', 'Lat',
    geo=True,
    size='Humidity',
    color='blue',
    alpha=0.5,
    tiles='OSM',
    frame_width=800,
    frame_height=600,
    title='City Humidity'
)

map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria (for example, humidity > 70 and max temp < 90)
criteria_df = city_data_df[(city_data_df['Humidity'] > 70) & (city_data_df['Max Temp'] < 90)]

# Drop any rows with null values
criteria_df.dropna(inplace=True)

# Display sample data
criteria_df.head()

C:\Users\kimmi\AppData\Local\Temp\ipykernel_12400\1374590329.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  criteria_df.dropna(inplace=True)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,qaqortoq,60.7167,-46.0333,39.2,98,100,1.09,GL,2024-06-08
2,2,adamstown,-25.0660,-130.1015,73.4,82,11,4.97,PN,2024-06-08
5,5,swan hill,-35.3378,143.5544,46.4,94,96,2.80,AU,2024-06-08
6,6,sorong,-0.8833,131.2500,77.0,91,100,3.53,ID,2024-06-08
10,10,port-aux-francais,-49.3500,70.2167,42.8,96,100,14.44,TF,2024-06-08


### Step 3: Create a new DataFrame called `hotel_df`.

In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,colchani,BO,-20.3000,-66.9333,33,
1,qaqortoq,GL,60.7167,-46.0333,98,
2,adamstown,PN,-25.0660,-130.1015,82,
3,happy valley-goose bay,CA,53.3168,-60.3315,46,
4,mogocha,RU,53.7333,119.7667,60,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel
radius = 5000  # Define the radius (in meters) within which to search for hotels
params = {
    "type": "lodging",  # Specify the type of place to search for (lodging for hotels)
    "radius": radius,   # Set the radius parameter for the search
    "limit": 1,         # Limit the number of results to 1 (finding the nearest hotel)
    "filter": "",       # Placeholder for filter parameter (if needed)
    "bias": "",         # Placeholder for bias parameter (if needed)
    "apiKey": geoapify_key  # Your Geoapify API key (make sure to import or define it)
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{radius}@{lng},{lat}"
    params["bias"] = f"point:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = response["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()


Starting hotel search
colchani - nearest hotel: No hotel found
qaqortoq - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
happy valley-goose bay - nearest hotel: No hotel found
mogocha - nearest hotel: No hotel found
swan hill - nearest hotel: No hotel found
sorong - nearest hotel: No hotel found
atascadero - nearest hotel: No hotel found
stanley - nearest hotel: No hotel found
iqaluit - nearest hotel: No hotel found
port-aux-francais - nearest hotel: No hotel found
ushuaia - nearest hotel: No hotel found
vorgashor - nearest hotel: No hotel found
puerto ayora - nearest hotel: No hotel found
kourou - nearest hotel: No hotel found
burang - nearest hotel: No hotel found
portland - nearest hotel: No hotel found
axim - nearest hotel: No hotel found
port alfred - nearest hotel: No hotel found
utrik - nearest hotel: No hotel found
dudinka - nearest hotel: No hotel found
umea - nearest hotel: No hotel found
yellowknife - nearest hotel: No hotel found
shannon - nearest h

,City,Country,Lat,Lng,Humidity,Hotel Name
0,colchani,BO,-20.3000,-66.9333,33,No hotel found
1,qaqortoq,GL,60.7167,-46.0333,98,No hotel found
2,adamstown,PN,-25.0660,-130.1015,82,No hotel found
3,happy valley-goose bay,CA,53.3168,-60.3315,46,No hotel found
4,mogocha,RU,53.7333,119.7667,60,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [18]:
%%capture --no-display

# Configure the map plot function with hover information
def create_dataset(df):
    dimensions = {
        'Longitude': 'Lng',
        'Latitude': 'Lat',
        'Hotel Name': 'Hotel Name',
        'City': 'City',
        'Country': 'Country'
    }
    points = gv.Points(df, kdims=['Lng', 'Lat'], vdims=list(dimensions.values()))
    return points.opts(
        title='Cities with Nearest Hotels',
        width=800,
        height=600,
        tools=['hover'],
        cmap='viridis',
        color='blue',
        size=10,
        scaling_factor=10000,
        hover_cols=['City', 'Hotel Name', 'Country']  # Specify hover information
    )

# Display the map plot
create_dataset(city_data_df)


DataError: Supplied data does not contain specified dimensions, the following dimensions were not found: ['Hotel Name']

PandasInterface expects tabular data, for more information on supported datatypes see http://holoviews.org/user_guide/Tabular_Datasets.html